In [1]:
import json
from tqdm.auto import tqdm

In [2]:
file_path = "../data/code_refinement/ref-train.jsonl"

In [3]:
data = [json.loads(line) for line in open(file_path)]

In [4]:
print(len(data))

150406


In [5]:
print(data[0].keys())

dict_keys(['old_hunk', 'oldf', 'hunk', 'comment', 'ids', 'repo', 'ghid', 'old', 'new', 'lang'])


In [7]:
print(data[0]['hunk'])

@@ -66,6 +66,9 @@ bool TransformationAddGlobalVariable::IsApplicable(
   if (message_.initializer_id()) {
     // An initializer is not allowed if the storage class is Workgroup.
     if (storage_class == SpvStorageClassWorkgroup) {
+      assert(false &&
+             "By construction this transformation should not have an "
+             "initializer when Workgroup storage class is used.");
       return false;
     }
     // The initializer id must be the id of a constant.  Check this with the



In [12]:
lines = data[0]['oldf'].split('\n')

In [17]:
print(lines[70])

    // The initializer id must be the id of a constant.  Check this with the


In [43]:
def apply_hunk_to_string(old_content, hunk):
    # Parsing the hunk header to get the starting line and the number of lines affected in the old and new files.
    hunk_lines = hunk.strip().split('\n')
    header = hunk_lines[0]
    old_info, new_info = header.split('@@ ')[1].split(' @@')[0].split()
    old_start, old_count = map(int, old_info[1:].split(','))
    new_start, new_count = map(int, new_info[1:].split(','))

    # Split the old content into lines, ensuring we don't lose any line including the first and last ones
    old_lines = old_content.split('\n')[:-1]

    # Initialize the new content with lines before the hunk
    new_lines = old_lines[:old_start - 1]

    # Apply the hunk changes
    old_index = old_start - 1
    for line in hunk_lines[1:]:
        if line.startswith('+'):
            # Line added
            new_lines.append(line[1:])
        elif line.startswith('-'):
            # Line deleted, so increment old_index to skip this line in the old file
            old_index += 1
        else:
            # Unchanged or context line, add it to new_lines and increment old_index
            new_lines.append(old_lines[old_index])
            old_index += 1

    # Add remaining lines after the hunk
    new_lines.extend(old_lines[old_index:])

    # Join the lines to form the new content
    new_content = '\n'.join(new_lines)
    return new_content

In [44]:
# Example usage
old_content = """some code before
 if (message_.initializer_id()) {
    if (storage_class == SpvStorageClassWorkgroup) {
       return false;
    }
 more code after
"""

hunk = """
@@ -2,6 +2,9 @@
 if (message_.initializer_id()) {
-    if (storage_class == SpvStorageClassWorkgroup) {
+      assert(false &&
+             "By construction this transformation should not have an "
+             "initializer when Workgroup storage class is used.");
       return false;
     }
"""

new_content = apply_hunk_to_string(old_content, hunk)
print(new_content)

some code before
 if (message_.initializer_id()) {
      assert(false &&
             "By construction this transformation should not have an "
             "initializer when Workgroup storage class is used.");
       return false;
    }
 more code after


In [50]:
old_code = data[1]['oldf']
hunk = data[1]['hunk']
new_code = apply_hunk_to_string(old_code, hunk)

In [62]:
n = 0
for i, d in enumerate(data):
    try:
        old_code = d['oldf']
        hunk = d['hunk']
        new_code = apply_hunk_to_string(old_code, hunk)
    except:
        print(i)
        n+=1
    if (i+1)%10000==0:
        print(f'##### Iteration {i+1} DONE #####')

160
538
834
918
945
1013
1077
1240
1296
1784
2608
3044
3535
3649
3656
3793
3817
4203
4370
4652
4973
5217
6150
6344
6369
6426
6753
6812
6864
7414
7869
8530
9107
9496
9967
##### Iteration 10000 DONE #####
10190
10485
10556
10947
11861
12035
12672
12753
13495
13903
14422
14494
14585
14855
15422
15892
15916
16106
17095
17160
17488
17655
17835
17912
17940
19059
19074
19169
19921
##### Iteration 20000 DONE #####
20062
20434
21104
21620
21923
22014
22046
22091
22631
22988
23125
23210
23252
23306
23658
24455
24793
25118
25275
25333
25583
25814
26178
26472
26784
27220
27818
27910
28119
29233
29345
29393
29777
29972
##### Iteration 30000 DONE #####
30025
30247
30451
31106
31761
32011
34602
34803
35020
35116
36342
36440
36470
37091
37605
37617
37943
38343
38820
39565
39836
39886
##### Iteration 40000 DONE #####
40063
40114
40724
40936
41025
42212
42255
42452
42599
42824
43033
44483
44572
44649
45079
45091
45456
45751
46648
46719
46773
46804
47149
47208
47504
47623
47799
47851
48246
48814
49421
##

In [64]:
print(f'Ratio of errors {n} / {len(data)}')

Ratio of errors 447 / 150406


In [80]:
# vizualize errors
j = 160
old_code = data[j]['oldf']
hunk = data[j]['hunk']
new_code = apply_hunk_to_string(old_code, hunk)

IndexError: list index out of range

In [81]:
print(hunk)

@@ -1,4 +0,0 @@
-SNAPSHOT_PAGE = {
-    'dashboard': True,
-    'project': True
-}
\ No newline at end of file



In [82]:
print(old_code)

SNAPSHOT_PAGE = {
    'dashboard': True,
    'project': True
}


In [83]:
print(data[j]['new'])

\ No newline at end of file


In [84]:
print(data[j]['old'])

-SNAPSHOT_PAGE = {
-    'dashboard': True,
-    'project': True
-}
\ No newline at end of file
